# Pytorch 103 - Neural Networks

For this unit we'll look at the use of `torch.nn` package.  
  
`nn` depends on `autograd` to define models and differentiate them.  
  
An `nn.Module` contains layers, and a method `fordward(input)` that returns the `output`

## An example

One of the best studied examples on Neural Networks (NNs) is a digit classification.  

It is a simple feed-forward network, this means it takes the input, feeds it through several layers one after the other and then finally gives the output.  

A typical training procedure for a NN is as follows:

1. Define the NN that has some learnable parameters (or weights).
2. ITerate over a dataset of inputs.
3. Process input throught the network.
4. Compute the loss (how far is the output from being correct).
5. Propagate gradients back into the network's parameters/
6. Update the weights of the network, typically using a simple update rule i.e, `weight = weoght - learning rate * gradient`

## Define the Network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# Let's create a network inherited from the nn.Module
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120) # 6 * 6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        
        # If the size is a square you can only specify a single number.
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x= self.fc3(x)
        
        return x
    def num_flat_features(self, x):
        # All dimensions except the batch dimension
        size = x.size()[1:]
        num_features = 1 
        for s in size:
            num_features *= s
            
        return num_features

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


Pytorch allows us to just define the `forward` function while the `backward` function is (where the gradiets are computed) is automatically generated for us using `autograd`.  

You can use any of the Tensor operations in the forward function.  
  
The learnable parameters of a model are teturned by `net.parameters()`

In [6]:
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


Let's try a random 32x32 input.  
Note: expected input size of this networ (LeNet) is 32x32.  
To use this net on the MNIST dataset please resize the images from the dataset to 32x32. 

In [11]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0177,  0.0869,  0.0676, -0.0341, -0.0022,  0.0586,  0.0467,  0.0869,
         -0.0425,  0.0264]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [12]:
net.zero_grad()
out.backward(torch.randn(1, 10))

TO BE CONTINUED